In [1]:
# Import libraries for simulation
import tensorflow as tf
import numpy as np

# Imports for visualization
import PIL.Image
from io import BytesIO
from IPython.display import Image, display

In [2]:
def DisplayFractal(a, fmt='jpeg'):
    """Display an array of iteration counts as a
       colorful picture of a fractal."""
    a_cyclic = (6.28*a/20.0).reshape(list(a.shape)+[1])
    img = np.concatenate([10+20*np.cos(a_cyclic),
                          30+50*np.sin(a_cyclic),
                          155-80*np.cos(a_cyclic)], 2)
    img[a==a.max()] = 0
    a = img
    a = np.uint8(np.clip(a, 0, 255))
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))

In [64]:
sess = tf.InteractiveSession()

Exception ignored in: <bound method BaseSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x12febd080>>
Traceback (most recent call last):
  File "/Users/abramo/anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 171, in __del__
    self.close()
  File "/Users/abramo/anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 976, in close
    self._default_session.__exit__(None, None, None)
  File "/Users/abramo/anaconda/envs/py35/lib/python3.5/contextlib.py", line 66, in __exit__
    next(self.gen)
  File "/Users/abramo/anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/framework/ops.py", line 3378, in get_controller
    % type(default))
AssertionError: Nesting violated for default stack of <class 'weakref'> objects


In [ ]:
# Use NumPy to create a 2D array of complex numbers on [-2,2]x[-2,2]

Y_MIN = -0.1
Y_MAX =  0.1

X_MIN = -1.6
X_MAX = -1.5

ZOOM = 10**3
H    = 1/(ZOOM * 10**3)

Y, X = np.mgrid[Y_MIN:Y_MAX:H, X_MIN:X_MAX:H]
Z = X+1j*Y

xs = tf.constant(Z.astype(np.complex64))
zs = tf.Variable(xs)
ns = tf.Variable(tf.zeros_like(xs, tf.float32))

tf.initialize_all_variables().run()

In [ ]:
# Compute the new values of z: z^2 + x
zs_ = zs*zs + xs

# Have we diverged with this new value?
not_diverged = tf.complex_abs(zs_) < 4

# Operation to update the zs and the iteration count.
#
# Note: We keep computing zs after they diverge! This
#       is very wasteful! There are better, if a little
#       less simple, ways to do this.
#
step = tf.group(
  zs.assign(zs_),
  ns.assign_add(tf.cast(not_diverged, tf.float32))
  )

In [59]:
for i in range(50): step.run()

In [60]:
DisplayFractal(ns.eval())